# Projet SER

In [1]:
import os
import numpy as np
import pandas as pd
import joblib
import json
import kaggle
import keras

from IPython.display import Audio
import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import  LabelEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score

from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

### 1. RAVDESS

In [2]:
Ravdess = "data_SIAKE/audio_speech_actors_01-24"

In [3]:
ravdess_directory_list = os.listdir(Ravdess)

file_emotion = []
file_path = []
for dir in ravdess_directory_list:
    # as their are 20 different actors in our previous directory we need to extract files for each actor.
    actor = os.listdir(Ravdess + '/' + dir)
    for file in actor:
        part = file.split('.')[0]
        part = part.split('-')
        
        if len(part) >= 3:
            # third part in each file represents the emotion associated to that file.
            file_emotion.append(int(part[2]))
            file_path.append(Ravdess + '/' + dir + '/' + file)

In [4]:
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Ravdess_df = pd.concat([emotion_df, path_df], axis=1)

# changing integers to actual emotions.
Ravdess_df.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
Ravdess_df.head()

,Emotions,Path
0,angry,data_SIAKE/audio_speech_actors_01-24/Actor_16/...
1,fear,data_SIAKE/audio_speech_actors_01-24/Actor_16/...
2,fear,data_SIAKE/audio_speech_actors_01-24/Actor_16/...
3,angry,data_SIAKE/audio_speech_actors_01-24/Actor_16/...
4,disgust,data_SIAKE/audio_speech_actors_01-24/Actor_16/...


### 2. Crema DataFrame

In [5]:
Crema = "data_SIAKE/AudioWAV"

In [6]:
crema_directory_list = os.listdir(Crema)

file_emotion = []
file_path = []

for file in crema_directory_list:
    # storing file paths
    file_path.append(Crema + '/'+  file)
    # storing file emotions
    part=file.split('_')
    
    if len(part) >= 3:
        if part[2] == 'SAD':
            file_emotion.append('sad')
        elif part[2] == 'ANG':
            file_emotion.append('angry')
        elif part[2] == 'DIS':
            file_emotion.append('disgust')
        elif part[2] == 'FEA':
            file_emotion.append('fear')
        elif part[2] == 'HAP':
            file_emotion.append('happy')
        elif part[2] == 'NEU':
            file_emotion.append('neutral')
        else:
            file_emotion.append('Unknown')
            
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Crema_df = pd.concat([emotion_df, path_df], axis=1)
Crema_df.head()

,Emotions,Path
0,angry,data_SIAKE/AudioWAV/1022_ITS_ANG_XX.wav
1,angry,data_SIAKE/AudioWAV/1037_ITS_ANG_XX.wav
2,neutral,data_SIAKE/AudioWAV/1060_ITS_NEU_XX.wav
3,neutral,data_SIAKE/AudioWAV/1075_ITS_NEU_XX.wav
4,disgust,data_SIAKE/AudioWAV/1073_IOM_DIS_XX.wav


### 3. TESS Dataframe

In [7]:
Tess = "data_SIAKE/TESS Toronto emotional speech set data"

In [8]:
tess_directory_list = os.listdir(Tess)

file_emotion = []
file_path = []

for dir in tess_directory_list:
    directories = os.listdir(Tess + '/' + dir)
    for file in directories:
        part = file.split('.')[0]
        split_part = part.split('_')
        
        # Check if the split result has the expected number of parts
        if len(split_part) >= 3:
            part = split_part[2]
            if part == 'ps':
                file_emotion.append('surprise')
            else:
                file_emotion.append(part)
            file_path.append(Tess + '/' + dir + '/' + file)

# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Tess_df = pd.concat([emotion_df, path_df], axis=1)
Tess_df.head()

,Emotions,Path
0,disgust,data_SIAKE/TESS Toronto emotional speech set d...
1,disgust,data_SIAKE/TESS Toronto emotional speech set d...
2,disgust,data_SIAKE/TESS Toronto emotional speech set d...
3,disgust,data_SIAKE/TESS Toronto emotional speech set d...
4,disgust,data_SIAKE/TESS Toronto emotional speech set d...


### Final dataframe with all the files paths

In [9]:
# creating Dataframe using all the 3 dataframes we created so far.
data_path = pd.concat([Ravdess_df, Crema_df, Tess_df], axis = 0)
data_path.to_csv("data_path.csv",index=False)
data_path.head()

,Emotions,Path
0,angry,data_SIAKE/audio_speech_actors_01-24/Actor_16/...
1,fear,data_SIAKE/audio_speech_actors_01-24/Actor_16/...
2,fear,data_SIAKE/audio_speech_actors_01-24/Actor_16/...
3,angry,data_SIAKE/audio_speech_actors_01-24/Actor_16/...
4,disgust,data_SIAKE/audio_speech_actors_01-24/Actor_16/...


In [67]:
path = pd.read_csv("data_path.csv")
path.head()

,Emotions,Path
0,angry,data_SIAKE/audio_speech_actors_01-24/Actor_16/...
1,fear,data_SIAKE/audio_speech_actors_01-24/Actor_16/...
2,fear,data_SIAKE/audio_speech_actors_01-24/Actor_16/...
3,angry,data_SIAKE/audio_speech_actors_01-24/Actor_16/...
4,disgust,data_SIAKE/audio_speech_actors_01-24/Actor_16/...


In [15]:
# List of feature names
feature_names = [
    'ZCR', 'Energy', 'SpectralCentroid', 'SpectralSpread', 'SpectralFlux',
    'MFCC1', 'MFCC2', 'MFCC3', 'MFCC4', 'MFCC5', 'MFCC6', 'MFCC7',
    'MFCC8', 'MFCC9', 'MFCC10', 'MFCC11', 'MFCC12', 'MFCC13',
    'Chroma1', 'Chroma2', 'Chroma3', 'Chroma4', 'Chroma5', 'Chroma6',
    'Chroma7', 'Chroma8', 'Chroma9', 'Chroma10', 'Chroma11', 'Chroma12'
]


In [19]:
X, Y= [], []
for path, emotion in zip(data_path.Path, data_path.Emotions):
    feature = get_features(path)
    X.append(feature)
    Y.append(emotion)

/Users/siakesophie/Local_Docs/ING_3/S9/Engineering_project/project/SERAPP/.env-serapp/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


In [22]:
len(X), len(Y), data_path.Path.shape

(11682, 11682, (11682,))

### Création du dataset

In [23]:
Features_simple = pd.DataFrame(X, columns=feature_names)
Features_simple['labels'] = Y
Features_simple.to_csv('features_simple.csv', index=False)
Features_simple.head()

,ZCR,Energy,SpectralCentroid,SpectralSpread,SpectralFlux,MFCC1,MFCC2,MFCC3,MFCC4,MFCC5,...,Chroma4,Chroma5,Chroma6,Chroma7,Chroma8,Chroma9,Chroma10,Chroma11,Chroma12,labels
0,0.204540,0.018793,3157.236579,2216.501199,2.351883,-440.286591,52.299297,-6.665863,4.278304,-15.808314,...,0.321975,0.346781,0.480067,0.331558,0.274953,0.354453,0.393035,0.393787,0.369280,angry
1,0.189815,0.012744,2744.419931,1984.251392,1.925617,-484.020691,52.436253,-8.613789,6.417563,-3.227553,...,0.285417,0.207637,0.242176,0.227869,0.242703,0.299185,0.388764,0.382405,0.378763,fear
2,0.200589,0.061085,3130.434636,2301.913172,2.341648,-315.505432,27.667685,-38.753201,0.213876,-14.901350,...,0.237158,0.168251,0.235008,0.320012,0.395711,0.443678,0.420142,0.433307,0.439407,fear
3,0.177156,0.040083,2874.637946,2220.399237,2.429970,-346.149780,52.521111,-14.139482,6.766320,-16.968788,...,0.367692,0.330350,0.368875,0.278672,0.218055,0.257016,0.376277,0.342528,0.312693,angry
4,0.175203,0.006512,2897.465696,2159.335761,2.339541,-532.369385,70.741547,-1.425078,11.303340,-9.632647,...,0.345111,0.322321,0.372758,0.350202,0.387294,0.438436,0.350818,0.331767,0.371133,disgust
